# Accessing data from an API

This notebook has two simple excerises demonstrating how to extract data from an [Application Programming Interface](https://en.wikipedia.org/wiki/API). An API is a tool for computers or applications to interact with one another. In our case, we'll be asking for data, and the API will return it. These systems can be complicated, but most of those we might use in data journalism are relatively simple.

#### Import our data tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

---

## Cat facts!

[Read the documentation](https://alexwohlbruck.github.io/cat-facts/docs/)

#### Get random facts

In [6]:
cat_df = pd.read_json(
    "https://cat-fact.herokuapp.com/facts/random?animal_type=cat&amount=500"
)

#### First five rows

In [7]:
cat_df.head()

,status,_id,user,text,__v,source,updatedAt,type,createdAt,deleted,used,sendDate
0,"{'verified': True, 'sentCount': 1}",58e008a30aac31001185ed0b,58e007480aac31001185ecef,"A cat's purr may be a form of self-healing, as it can be a sign of nervousness as well as contentment.",0.0,user,2020-08-23T20:20:01.611Z,cat,2018-02-24T21:20:04.261Z,False,0.0,NaN
1,"{'verified': None, 'sentCount': 0}",61d555f742bf74fddcf7f766,61bf8fbc99846b07e4af8af2,ПЫЩЩЩЩЩЩЩЩЩЩЩЩЩЩЩЩ.,0.0,NaN,2022-01-05T08:25:27.599Z,cat,2022-01-05T08:25:27.599Z,False,NaN,NaN
2,"{'verified': True, 'sentCount': 1}",591f98803b90f7150a19c206,5a9ac18c7478810ea6c06381,Cats can predict earthquakes. We humans are not 100% sure how they do it. There are several different theories.,0.0,api,2020-08-23T20:20:01.611Z,cat,2018-01-04T01:10:54.673Z,False,0.0,NaN
3,"{'verified': True, 'sentCount': 1}",591f98783b90f7150a19c1e0,5a9ac18c7478810ea6c06381,"A cat has a total of 24 whiskers, 4 rows of whiskers on each side. The upper two rows can move independently of the bottom two rows.",0.0,api,2020-08-23T20:20:01.611Z,cat,2018-04-21T20:20:02.287Z,False,0.0,NaN
4,"{'verified': True, 'sentCount': 1}",5a4d75e46ef087002174c284,5a9ac18c7478810ea6c06381,"Cats have about 130,000 hairs per square inch (20,155 hairs per square centimeter).",0.0,user,2020-08-23T20:20:01.611Z,cat,2018-04-05T20:20:05.171Z,False,0.0,NaN


#### How many records? 

In [8]:
len(cat_df)

500

#### What's the first fact?

In [9]:
cat_df["text"][0]

"A cat's purr may be a form of self-healing, as it can be a sign of nervousness as well as contentment."

#### Exctract the nested json inside the `status` column

In [20]:
cat_df[["verified", "sentCount", "feedback"]] = pd.json_normalize(cat_df["status"])

#### Slim the dataframe

In [21]:
cat_df_slim = cat_df[["_id", "text", "createdAt", "verified"]].copy()

In [22]:
cat_df_slim.head()

,_id,text,createdAt,verified
0,58e008a30aac31001185ed0b,"A cat's purr may be a form of self-healing, as it can be a sign of nervousness as well as contentment.",2018-02-24T21:20:04.261Z,True
1,61d555f742bf74fddcf7f766,ПЫЩЩЩЩЩЩЩЩЩЩЩЩЩЩЩЩ.,2022-01-05T08:25:27.599Z,None
2,591f98803b90f7150a19c206,Cats can predict earthquakes. We humans are not 100% sure how they do it. There are several different theories.,2018-01-04T01:10:54.673Z,True
3,591f98783b90f7150a19c1e0,"A cat has a total of 24 whiskers, 4 rows of whiskers on each side. The upper two rows can move independently of the bottom two rows.",2018-04-21T20:20:02.287Z,True
4,5a4d75e46ef087002174c284,"Cats have about 130,000 hairs per square inch (20,155 hairs per square centimeter).",2018-04-05T20:20:05.171Z,True


#### Just the verified facts, pls

In [47]:
verified_df = cat_df_slim[cat_df_slim["verified"] == True].copy()

In [48]:
len(verified_df)

299

#### Find facts that mentions specific words? 

In [49]:
verified_df[verified_df["text"].str.lower().str.contains("dog|food|toys")].head()

,_id,text,createdAt,verified
44,5c60999ae549020014533038,"Cats love playing with yarn, ribbons, and fishing-rode style toys. However, cats should not be left alone with anything that they could get tangled in.",2019-02-10T21:37:30.093Z,True
51,58e00a090aac31001185ed16,Cats make more than 100 different sounds whereas dogs make around 10.,2018-02-11T21:20:03.745Z,True
61,58e007cc0aac31001185ecf5,Cats are the most popular pet in the United States: There are 88 million pet cats and 74 million dogs.,2018-03-01T21:20:02.713Z,True
82,5887e1d85c873e0011036889,Cats make about 100 different sounds. Dogs make only about 10.,2018-01-15T21:20:00.003Z,True
88,591f98008dec2e14e3c20b0b,"Cats have 30 teeth (12 incisors, 10 premolars, 4 canines, and 4 molars), while dogs have 42. Kittens have baby teeth, which are replaced by permanent teeth around the age of 7 months.",2018-01-04T01:10:54.673Z,True


#### Find the oldest fact? 

In [56]:
verified_df.sort_values("createdAt", ascending=False).head()

,_id,text,createdAt,verified,date
385,5daa192179186100154250c4,"GitHub is a cloud source version control system where its mascot is an octocat, an anthropomorphized cat with five tentacles.",2019-10-18T19:57:21.696Z,True,2019-10-18
484,5d9c556168a764001553b382,"A cat has 244 bones in its entire body—even more than a human, who only has 206 bones.",2019-10-08T09:22:41.032Z,True,2019-10-08
43,5d38be370f1c57001592f158,Kittens typically begin to engage in playful behavior at around four weeks of age.,2019-07-24T20:23:19.615Z,True,2019-07-24
414,5d38bc0c0f1c57001592f150,"If your cat licks from the toilet, bathtub, or sink, she may be telling you something about the water bowl. Make sure it's clean, replace the water regularly to keep it from getting stale or developing a film on top, and if necessary, get a new bowl. Some cats prefer glass bowls to plastic.",2019-07-24T20:14:04.986Z,True,2019-07-24
290,5d38bbb20f1c57001592f14f,"American Curl kittens are born with straight ears, but the ears begin to curl back after just a few days.",2019-07-24T20:12:34.736Z,True,2019-07-24


In [51]:
verified_df["date"] = pd.to_datetime(verified_df["createdAt"]).dt.strftime("%Y-%m-%d")

In [52]:
verified_df.head()

,_id,text,createdAt,verified,date
0,58e008a30aac31001185ed0b,"A cat's purr may be a form of self-healing, as it can be a sign of nervousness as well as contentment.",2018-02-24T21:20:04.261Z,True,2018-02-24
2,591f98803b90f7150a19c206,Cats can predict earthquakes. We humans are not 100% sure how they do it. There are several different theories.,2018-01-04T01:10:54.673Z,True,2018-01-04
3,591f98783b90f7150a19c1e0,"A cat has a total of 24 whiskers, 4 rows of whiskers on each side. The upper two rows can move independently of the bottom two rows.",2018-04-21T20:20:02.287Z,True,2018-04-21
4,5a4d75e46ef087002174c284,"Cats have about 130,000 hairs per square inch (20,155 hairs per square centimeter).",2018-04-05T20:20:05.171Z,True,2018-04-05
5,591f98803b90f7150a19c23e,Cats come back to full alertness from the sleep state faster than any other creature.,2018-06-01T20:20:02.011Z,True,2018-06-01


#### Most recent verified fact?

In [55]:
verified_df.sort_values("date", ascending=False).head()

,_id,text,createdAt,verified,date
385,5daa192179186100154250c4,"GitHub is a cloud source version control system where its mascot is an octocat, an anthropomorphized cat with five tentacles.",2019-10-18T19:57:21.696Z,True,2019-10-18
484,5d9c556168a764001553b382,"A cat has 244 bones in its entire body—even more than a human, who only has 206 bones.",2019-10-08T09:22:41.032Z,True,2019-10-08
414,5d38bc0c0f1c57001592f150,"If your cat licks from the toilet, bathtub, or sink, she may be telling you something about the water bowl. Make sure it's clean, replace the water regularly to keep it from getting stale or developing a film on top, and if necessary, get a new bowl. Some cats prefer glass bowls to plastic.",2019-07-24T20:14:04.986Z,True,2019-07-24
12,5d38b5b50f1c57001592f131,"Andy Warhol amassed quite the collection of feline friends during his lifetime, beginning with a Siamese cat named Hester that was given to him by actress Gloria Swanson. By breeding Hester with a cat named Sam, Warhol ended up with multiple litters of kittens, at one point housing 25 cats in his Upper East Side townhouse in NYC.",2019-07-24T19:47:01.970Z,True,2019-07-24
290,5d38bbb20f1c57001592f14f,"American Curl kittens are born with straight ears, but the ears begin to curl back after just a few days.",2019-07-24T20:12:34.736Z,True,2019-07-24


---

## Dad jokes!

[Read the documentation](https://icanhazdadjoke.com/api#fetch-a-random-dad-joke)

#### Give the request headers so the API knows how to answer it

In [58]:
headers = {
    "Accept": "application/json",
}

#### Get a response from the API in the format we requested

In [59]:
response = requests.get("https://icanhazdadjoke.com/search?page=1", headers=headers)

#### What comes back?

In [60]:
response.json()

{'current_page': 1,
 'limit': 20,
 'next_page': 2,
 'previous_page': 1,
 'results': [{'id': '0189hNRf2g',
   'joke': "I'm tired of following my dreams. I'm just going to ask them where they are going and meet up with them later."},
  {'id': '08EQZ8EQukb',
   'joke': "Did you hear about the guy whose whole left side was cut off? He's all right now."},
  {'id': '08xHQCdx5Ed',
   'joke': 'Why didn’t the skeleton cross the road? Because he had no guts.'},
  {'id': '0DQKB51oGlb',
   'joke': "What did one nut say as he chased another nut?  I'm a cashew!"},
  {'id': '0DtrrOZDlyd',
   'joke': "Chances are if you' ve seen one shopping center, you've seen a mall."},
  {'id': '0LuXvkq4Muc',
   'joke': "I knew I shouldn't steal a mixer from work, but it was a whisk I was willing to take."},
  {'id': '0ga2EdN7prc',
   'joke': 'How come the stadium got hot after the game? Because all of the fans left.'},
  {'id': '0oO71TSv4Ed',
   'joke': 'Why was it called the dark ages? Because of all the knights.

#### What's the limit per API call? 

In [67]:
response.json()

{'current_page': 1,
 'limit': 20,
 'next_page': 2,
 'previous_page': 1,
 'results': [{'id': '0189hNRf2g',
   'joke': "I'm tired of following my dreams. I'm just going to ask them where they are going and meet up with them later."},
  {'id': '08EQZ8EQukb',
   'joke': "Did you hear about the guy whose whole left side was cut off? He's all right now."},
  {'id': '08xHQCdx5Ed',
   'joke': 'Why didn’t the skeleton cross the road? Because he had no guts.'},
  {'id': '0DQKB51oGlb',
   'joke': "What did one nut say as he chased another nut?  I'm a cashew!"},
  {'id': '0DtrrOZDlyd',
   'joke': "Chances are if you' ve seen one shopping center, you've seen a mall."},
  {'id': '0LuXvkq4Muc',
   'joke': "I knew I shouldn't steal a mixer from work, but it was a whisk I was willing to take."},
  {'id': '0ga2EdN7prc',
   'joke': 'How come the stadium got hot after the game? Because all of the fans left.'},
  {'id': '0oO71TSv4Ed',
   'joke': 'Why was it called the dark ages? Because of all the knights.

#### How many total jokes? 

In [66]:
response.json()["total_jokes"]

649

#### How many pages of 20 jokes? 

In [68]:
response.json()["total_pages"]

33

#### Ok, just the jokes

In [64]:
jokes_df = pd.DataFrame(response.json()["results"])

#### How many records?

In [65]:
len(jokes_df)

20

#### Get all the jokes with a loop

In [72]:
data_pages = []

for r in range(0, 34):
    data_pages.append(
        pd.DataFrame(
            requests.get(
                f"https://icanhazdadjoke.com/search?page={r}", headers=headers
            ).json()["results"]
        )
    )

jokes_df = pd.concat(data_pages).reset_index(drop=True)

#### How many? 

In [73]:
len(jokes_df)

669

In [74]:
jokes_df.head()

,id,joke
0,0189hNRf2g,I'm tired of following my dreams. I'm just going to ask them where they are going and meet up with them later.
1,08EQZ8EQukb,Did you hear about the guy whose whole left side was cut off? He's all right now.
2,08xHQCdx5Ed,Why didn’t the skeleton cross the road? Because he had no guts.
3,0DQKB51oGlb,What did one nut say as he chased another nut? I'm a cashew!
4,0DtrrOZDlyd,"Chances are if you' ve seen one shopping center, you've seen a mall."


In [75]:
jokes_df.to_csv("../../data/processed/dad-jokes.csv", index=False)